In [1]:
%%capture
!pip install pyloudnorm
!pip install pysndfx
!pip install speechbrain
!pip install transformers
!pip install speechbrain.pretrained
!pip install torchmetrics
!pip install wandb

In [2]:
import os
from multiprocessing import Process
import subprocess
import glob

import random
import warnings
import tarfile

import numpy as np
import pandas as pd
import pyloudnorm as pyln
import soundfile as sf
import tqdm as tqdm_module
import zipfile

import tqdm.contrib.concurrent
import functools
from pysndfx import AudioEffectsChain

import librosa

import torchaudio
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torchaudio.transforms as T
import torchmetrics

from IPython.display import Audio

import speechbrain as sb
from speechbrain.dataio.dataio import read_audio
from IPython.display import Audio
from speechbrain.inference.separation import SepformerSeparation as separator

from torchmetrics.audio import ScaleInvariantSignalNoiseRatio
from torchmetrics.audio import ScaleInvariantSignalDistortionRatio
import wandb

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
! wandb login c07f9b9363b1d2736cf24c01a4747e245909f2cc

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
!unzip /content/drive/MyDrive/LibriMix.zip

Streaming output truncated to the last 5000 lines.
  inflating: wav8k/max/test/mix_both/1995-1826-0023_8463-294828-0020.wav  
  inflating: wav8k/max/test/mix_both/5142-36377-0022_4992-23283-0019.wav  
  inflating: wav8k/max/test/mix_both/4446-2271-0004_3570-5694-0002.wav  
  inflating: wav8k/max/test/mix_both/7021-79740-0000_1320-122612-0010.wav  
  inflating: wav8k/max/test/mix_both/2830-3980-0031_1995-1837-0003.wav  
  inflating: wav8k/max/test/mix_both/260-123286-0005_1580-141084-0026.wav  
  inflating: wav8k/max/test/mix_both/6829-68769-0049_260-123288-0017.wav  
  inflating: wav8k/max/test/mix_both/237-134493-0011_7176-92135-0024.wav  
  inflating: wav8k/max/test/mix_both/2300-131720-0040_61-70970-0027.wav  
  inflating: wav8k/max/test/mix_both/4507-16021-0024_5639-40744-0038.wav  
  inflating: wav8k/max/test/mix_both/7127-75946-0018_2300-131720-0008.wav  
  inflating: wav8k/max/test/mix_both/1995-1837-0025_5142-33396-0014.wav  
  inflating: wav8k/max/test/mix_both/4446-2273-0008_

In [7]:
!unzip /content/drive/MyDrive/speech_task2_Q3.zip

Archive:  /content/drive/MyDrive/speech_task2_Q3.zip
  inflating: Speech_task2_Q3/extra_requirements.txt  
  inflating: Speech_task2_Q3/gradioo.py  
  inflating: Speech_task2_Q3/prepare_data.py  
  inflating: Speech_task2_Q3/Speech_assgn_task2_Q3.ipynb  
  inflating: Speech_task2_Q3/speech_assgn_task2_q3.py  
  inflating: Speech_task2_Q3/Trained_model/trained_separator.pth  
  inflating: Speech_task2_Q3/separation/dynamic_mixing.py  
  inflating: Speech_task2_Q3/separation/prepare_data.py  
  inflating: Speech_task2_Q3/separation/preprocess_dynamic_mixing.py  
  inflating: Speech_task2_Q3/separation/README.md  
  inflating: Speech_task2_Q3/separation/train.py  
  inflating: Speech_task2_Q3/separation/hparams/convtasnet.yaml  
  inflating: Speech_task2_Q3/separation/hparams/dprnn.yaml  
  inflating: Speech_task2_Q3/separation/hparams/resepformer.yaml  
  inflating: Speech_task2_Q3/separation/hparams/sepformer-conformerintra.yaml  
  inflating: Speech_task2_Q3/separation/hparams/sepforme

In [8]:
!pip install -r ../content/Speech_task2_Q3/extra_requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mir-eval: filename=mir_eval-0.6-py3-none-any.whl size=96498 sha256=0f20b8e2c8e7e4421253c63f962079a6e1d2bb2c6f89341e024f1dcdded8b04a
  Stored in directory: /root/.cache/pip/wheels/76/7f/01/3506e105a83b8e9cfad28631afed4caf0f3569a51ab6ed06f1
Successfully built mir-eval


In [10]:
!python /content/Speech_task2_Q3/separation/train.py /content/Speech_task2_Q3/separation/hparams/sepformer-customdataset.yaml --data_folder /content

wandb: Currently logged in as: m23csa011 (kushal1506). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /content/wandb/run-20240412_165513-tvvfp2g5
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run hardy-terrain-14
wandb: ⭐️ View project at https://wandb.ai/kushal1506/FineTune
wandb: 🚀 View run at https://wandb.ai/kushal1506/FineTune/runs/tvvfp2g5
speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: results/sepformer-custom/1234
speechbrain.core - Info: precision arg from hparam file is used
speechbrain.core - Info: noprogressbar arg from hparam file is used
speechbrain.core - Gradscaler enabled: False. Using precision: fp32.
speechbrain.core - 6.7M trainable parameters in Separation
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in cu